In [1]:
# Call libraries for data cleaning and data pushing
import pandas as pd
import pymongo

In [2]:
# List of states' abbreviation in alphabetical order
states_abbrev = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [3]:
# List of states in alphabetical order
states = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]


In [4]:
# Create a dataframe for the states and its' abbreviations
Template_State = pd.DataFrame({"States":states,
                              "States_ab":states_abbrev});

In [5]:
# Import csv files for data merging and cleaning
Capital_Data = pd.read_csv("../data/50_states.csv")
Labor_Force_Data = pd.read_csv("../data/Labor Force Data.csv")

In [6]:
# Merge both datasets 
Combined_Data = Template_State.merge(Capital_Data, left_on = "States", right_on="GeoName")
Combined_Data = Combined_Data.merge(Labor_Force_Data, left_on = "States", right_on="State",how="inner")

# Drop unnecssary data
Drop_Data = ["GeoName","LineCode","2017:Q1","2017:Q2","2017:Q3","State"]
Combined_Data = Combined_Data.drop(columns = Drop_Data)

In [12]:
Starbucks_Locations = pd.read_csv("../data/us_directory.csv")
Starbucks_Locations = Starbucks_Locations.merge(Template_State, left_on = "State/Province", right_on="States_ab")

Drop_Data = ["Unnamed: 0","Brand","Country","Postcode","Phone Number","Timezone","States_ab"]
Starbucks_Locations = Starbucks_Locations.drop(columns = Drop_Data)

Combined_Data.head()

,States,States_ab,Description,2017:Q4,Yearly Avg,Labor Force
0,Alabama,AL,Per capita personal income (dollars) 2/,41079,40459.25,2181333
1,Alaska,AK,Per capita personal income (dollars) 2/,57527,56794.75,358859
2,Arizona,AZ,Per capita personal income (dollars) 2/,43128,42481.75,3368789
3,Arkansas,AR,Per capita personal income (dollars) 2/,42011,41517.50,1350866
4,California,CA,Per capita personal income (dollars) 2/,61474,60152.50,19252711


In [8]:
# Create a dictionary for upcoming api to insert into MongoDB
API_Data = {}

States = Combined_Data["States"].tolist()
Labor_Force = Combined_Data["Labor Force"].tolist()
Personal_Income_Capital = Combined_Data["2017:Q4"].tolist()

In [9]:
for i in range(len(Combined_Data)):
    API_Data[States[i]] = {"Labor Force":Labor_Force[i],
                          "Personal Income Capital":Personal_Income_Capital[i]}

In [10]:
List_Of_Store_Numbers = []

#For Each state
for state in states:
    #Find a dataframe with all the starbucks in that state
    Locations_in_State = Starbucks_Locations[Starbucks_Locations["States"]==state]
    API_Data[state]["State"] = state
    API_Data[state]["Store Count"] = int(Locations_in_State["Store Number"].count())
    
    #Creates a dictionary entry called "Stores" in each state
    API_Data[state]["Stores"] = {}
    Stores = API_Data[state]["Stores"]
    List_Of_Store_Numbers.append(state)
    
    # For every store in "Locations_in_State" dataframe
    for store in Locations_in_State["Store Number"]:
        # Create a dictionary entry for the store number
        Stores[store] = {}
        Store_Number = Stores[store]
        # Finds the one data row corresponding to the store number
        Store_Information = Locations_in_State[Locations_in_State["Store Number"]==store]
        # Creates details for each store
        Store_Number["Ownership Type"] = Store_Information.iloc[0,2]
        Store_Number["Street Address"] = Store_Information.iloc[0,3]
        Store_Number["City"] = Store_Information.iloc[0,4]
        Store_Number["State"] = Store_Information.iloc[0,5]
        Store_Number["Longitude"] = Store_Information.iloc[0,6]
        Store_Number["Latitude"] = Store_Information.iloc[0,7]
    
 

#Import to MongoDB with PyMongo


In [11]:
# Create a link between Mongodb and python
conn = "mongodb://localhost:27017"
# Create a client so we can interact with mongodb
client = pymongo.MongoClient(conn)

# Call up StarbucksDB database
Starbucks_Information_db = client.StarbucksDB

# Call up Information collection
Information = Starbucks_Information_db.Information

# Insert API_Data to information collection
Information.update({},API_Data,upsert=True)

List_Of_Stores = {"Store":Starbucks_Locations["Store Number"].tolist()}

Store_List = Starbucks_Information_db.Store_List
Store_List.update({},List_Of_Stores,upsert=True)


C:\Users\Abraham\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  del sys.path[0]
C:\Users\Abraham\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.


{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}